# Managing State

到目前为止，我们已经学习了如何构建多智能体系统中的核心组件——如 Agents、Teams 和 Termination Conditions。
在真实应用中（尤其是 Web 应用），我们经常需要：

保存这些组件的运行状态到磁盘，以及

在之后重新加载这些状态，以便恢复中断的会话或继续未完成的任务。

这是因为许多 Web 应用是无状态的（stateless）：每次请求不会保留上次运行的内存数据。为了让系统“记得”对话进展，我们必须使用持久化机制保存组件状态。

🧠 在本节中，你将学到：

✅ 如何保存 agent、team、termination condition 的当前状态；

✅ 如何从保存的数据中重新加载并恢复运行；

✅ 为什么这对构建持续会话、异步交互和前端集成特别关键。

## 配置 Client

In [3]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-8B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称，硅基流动平台上 Qwen 3-8B 模型
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True           
    },
)

## Saving and Loading Agents

我们可以通过在 AssistantAgent 上调用 `.save_state()` 方法，来获取该智能体的状态信息。

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# 假设 model_client 已在前面初始化
model_client = model_client

assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)

# 在脚本中运行时请用 asyncio.run(...) 包裹主函数
response = await assistant_agent.on_messages(
    [TextMessage(content="Write a 3 line poem on lake tangayika", source="user")], CancellationToken()
)

print(response.chat_message)
await model_client.close()

source='assistant_agent' models_usage=RequestUsage(prompt_tokens=29, completion_tokens=452) metadata={} content="\n\nLake Tanganyika, a rift's sapphire gem,  \nWhere ancient tides whisper secrets to the sun,  \nLife thrives in depths, a world untouched by time." type='TextMessage'


In [5]:
agent_state = await assistant_agent.save_state()
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a 3 line poem on lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': "\n\nLake Tanganyika, a rift's sapphire gem,  \nWhere ancient tides whisper secrets to the sun,  \nLife thrives in depths, a world untouched by time.", 'thought': '\nOkay, the user wants a three-line poem about Lake Tanganyika. Let me start by recalling some key facts about the lake. It\'s one of the largest lakes in the world by volume, located in Africa, specifically in the East African Rift. I should mention its size or depth maybe. Also, it\'s home to a lot of endemic species, so that\'s a unique point. The surrounding countries are Tanzania, Burundi, Democratic Republic of Congo, and Zambia. Maybe include something about the countries or the local culture? Also, the lake is known for its clear waters and the presence of certain fish like sardines and tilapia. I should make it concise but vivid. L

In [6]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient

load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY") # 读取你的 OPENAI API key

# 重新初始化 OpenAIChatCompletionClient 客户端, 因为上面已经 close client 了
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True           
    },
)


new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)
await new_assistant_agent.load_state(agent_state)

# 在脚本中运行时请用 asyncio.run(...) 包裹主函数
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the last line of the previous poem you wrote", source="user")], CancellationToken()
)
print(response.chat_message)
await model_client.close()

source='assistant_agent' models_usage=RequestUsage(prompt_tokens=86, completion_tokens=209) metadata={} content='\n\nThe last line of the previous poem was:  \n**"Life thrives in depths, a world untouched by time."**  \n\nLet me know if you\'d like to expand or refine it! 🌊' type='TextMessage'


对于 AssistantAgent 来说，它的状态（state）主要包括其所使用的 模型上下文（model_context），也就是：

- 当前使用的模型（如 GPT-4、GPT-3.5）

- 系统消息（system_message）

- 聊天历史（如有）

- 工具、token usage 等上下文设置

🧩 自定义智能体：建议重写 `save_state()` 和 `load_state()`
如果你实现了自己的 agent（继承自 BaseChatAgent 或其他基类），请注意：

💡 默认的 `save_state()` 和 `load_state()` 是空实现，不会自动保存任何状态！

## Saving and Loading Teams

在多智能体系统中，保存团队（Team）的状态尤为关键，因为它能同时保存每个 agent 的状态、终止条件、对话历史等完整上下文。

**方法说明：**

- `team.save_state()`：返回一个包含团队结构和每个 agent 状态的字典，可序列化为 JSON。

- `team.load_state(state)`：从字典（或 JSON 数据）中还原团队的完整状态。

当你调用 `team.save_state()` 时，它会递归调用团队中所有 agent 的 save_state() 方法，一起打包保存。

In [7]:
import os
from dotenv import load_dotenv
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# 加载环境变量，获取 API 密钥
load_dotenv()
siliconflow_api_key = os.getenv("SILICONFLOW_API_KEY")  # 读取你的 API key

# 初始化 OpenAIChatCompletionClient 客户端，连接到硅基流动平台的 Qwen3-14B 模型
model_client = OpenAIChatCompletionClient(
    model="Qwen/Qwen3-14B",                # 指定要调用的模型名称
    base_url="https://api.siliconflow.cn/v1",  # 硅基流动平台的 API 访问地址
    api_key=siliconflow_api_key,  # 你的 API 密钥
    model_info={                        
        "family": "qwen",              
        "context_length": 8192,        
        "max_output_tokens": 2048,     
        "tool_choice_supported": True, 
        "tool_choice_required": False,  
        "structured_output": True,     
        "vision": False,                
        "function_calling": True,      
        "json_output": True           
    },
)

# 定义一个助手智能体
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client=model_client,
)

# 创建一个团队，设置最大消息数为2，达到后自动终止
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# 运行团队任务，并将消息流式输出到控制台
stream = agent_team.run_stream(task="Write a beautiful poem 3-line about lake tangayika")
await Console(stream)

# 保存团队的当前状态（包括 agent 状态、对话历史、终止条件等）
team_state = await agent_team.save_state()

---------- TextMessage (user) ----------
Write a beautiful poem 3-line about lake tangayika
---------- ThoughtEvent (assistant_agent) ----------

Okay, the user wants a beautiful three-line poem about Lake Tanganyika. Let me start by recalling what I know about Lake Tanganyika. It's one of the largest lakes in the world, located in East Africa, shared by several countries. It's known for its clear waters and the fact that it's a UNESCO World Heritage Site. Also, there's a lot of biodiversity there, including unique fish species.

Now, I need to capture the essence of the lake in just three lines. The first line should probably set the scene, maybe talking about its size or beauty. Words like "giant mirror," "sapphire," or "ancient" come to mind. The second line could mention the wildlife or the surrounding environment. Maybe something about the fish or the mountains around it. The third line should evoke a feeling or a lasting image, perhaps about timelessness or tranquility. I should 

如果我们重置团队（相当于重新实例化团队），然后提出问题“你写的诗的最后一句是什么？”，我们会发现团队无法完成这个任务，因为它无法引用之前的运行记录。

In [8]:
await agent_team.reset()
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)

---------- TextMessage (user) ----------
What was the last line of the poem you wrote?
---------- ThoughtEvent (assistant_agent) ----------

Okay, the user is asking for the last line of the poem I wrote. Wait, did I write a poem before?

Let me check the conversation history. Hmm, the user just started a new chat. There's no previous poem mentioned. Maybe they're referring to a poem I generated in a previous interaction that isn't recorded here. But in this context, I don't have any prior poems to reference.

I should clarify that I didn't write a poem in this conversation. I need to respond politely, letting them know I can help create one if they want. Maybe offer to write a poem for them now. Keep the tone friendly and helpful.

---------- TextMessage (assistant_agent) ----------


I didn't write a poem in our conversation yet! But I'd be happy to create one for you if you'd like. What kind of poem would you like me to write?


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote?', type='TextMessage'), ThoughtEvent(source='assistant_agent', models_usage=None, metadata={}, content="\nOkay, the user is asking for the last line of the poem I wrote. Wait, did I write a poem before?\n\nLet me check the conversation history. Hmm, the user just started a new chat. There's no previous poem mentioned. Maybe they're referring to a poem I generated in a previous interaction that isn't recorded here. But in this context, I don't have any prior poems to reference.\n\nI should clarify that I didn't write a poem in this conversation. I need to respond politely, letting them know I can help create one if they want. Maybe offer to write a poem for them now. Keep the tone friendly and helpful.\n", type='ThoughtEvent'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=171), metadata={}, content="\

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote?', type='TextMessage'), ThoughtEvent(source='assistant_agent', models_usage=None, metadata={}, content="\nOkay, the user is asking for the last line of the poem I wrote. Let me check the conversation history. Wait, looking back, I don't remember writing any poem in our previous messages. The user might be referring to a poem I mentioned earlier, but I need to confirm.\n\nI should respond by letting them know I haven't written a poem yet. Maybe they thought I did, but I need to be clear. Also, I can offer to write one if they want. But I need to make sure I'm not assuming anything. Let me phrase it politely and open-ended to invite them to ask for a poem if that's what they need.\n", type='ThoughtEvent'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=172), metadata={}, content="\n\nI haven't written a 

接下来，我们加载团队的状态并提出同样的问题。我们可以看到，团队能够准确地返回它之前所写诗歌的最后一句。


In [9]:
print(team_state)  # 打印团队当前保存的状态（可序列化为 JSON，便于持久化存储）

# 加载团队状态，实现断点续聊或会话恢复
await agent_team.load_state(team_state)

# 重新运行团队任务，此时团队已恢复之前的对话历史和上下文
stream = agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)

{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'assistant_agent': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a beautiful poem 3-line about lake tangayika', 'source': 'user', 'type': 'UserMessage'}, {'content': '\n\nLake Tanganyika, a sapphire scar across Africa’s heart,  \nWhere cichlids weave sunlit tales in waters deep and dark,  \nTime whispers through waves, cradling myths the earth forgot.', 'thought': '\nOkay, the user wants a beautiful three-line poem about Lake Tanganyika. Let me start by recalling what I know about Lake Tanganyika. It\'s one of the largest lakes in the world, located in East Africa, shared by several countries. It\'s known for its clear waters and the fact that it\'s a UNESCO World Heritage Site. Also, there\'s a lot of biodiversity there, including unique fish species.\n\nNow, I need to capture the essence of the lake in 

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last line of the poem you wrote?', type='TextMessage'), ThoughtEvent(source='assistant_agent', models_usage=None, metadata={}, content='\nOkay, the user is asking for the last line of the poem I wrote about Lake Tanganyika. Let me check the previous response.\n\nThe poem was three lines. The first line describes the lake as a sapphire scar, the second mentions cichlids and the environment, and the third line talks about time whispering through waves and myths. \n\nThe user wants the last line. I need to make sure I didn\'t make any mistake in recalling. The last line is "Time whispers through waves, cradling myths the earth forgot." \n\nWait, maybe they need confirmation. Let me recheck. Yes, the third line is the last one. The user might be remembering the poem or using it for something, like a school project or a presentation. They could be verifying the content or looking for inspir

## Persisting State (File or Database)

在许多情况下，我们可能希望将团队的状态持久化到磁盘（或数据库）中，并在之后加载回来。状态是一个可以序列化为文件或写入数据库的字典对象。

我已编写了 `tea,_state.json` 存储在 `./resurces` 

In [10]:
import json

# 保存团队状态到磁盘（序列化为 JSON 文件）
with open("resources/team_state.json", "w", encoding="utf-8") as f:
    json.dump(team_state, f, ensure_ascii=False, indent=2)

# 从磁盘加载团队状态（反序列化）
with open("resources/team_state.json", "r", encoding="utf-8") as f:
    team_state = json.load(f)

# 重新创建团队对象（注意要用新的团队实例）
new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# 加载团队状态，实现断点续聊或会话恢复
await new_agent_team.load_state(team_state)

# 重新运行团队任务，此时团队已恢复之前的对话历史和上下文
stream = new_agent_team.run_stream(task="What was the last line of the poem you wrote?")
await Console(stream)

# 关闭模型客户端，释放资源
await model_client.close()

---------- TextMessage (user) ----------
What was the last line of the poem you wrote?
---------- ThoughtEvent (assistant_agent) ----------

Okay, the user is asking for the last line of the poem I wrote about Lake Tanganyika. Let me check the history.

In the previous message, I provided a three-line poem. The lines were:

1. Lake Tanganyika, a sapphire scar across Africa’s heart,  
2. Where cichlids weave sunlit tales in waters deep and dark,  
3. Time whispers through waves, cradling myths the earth forgot.  

So the last line is "Time whispers through waves, cradling myths the earth forgot." The user might be confirming the poem's ending or using it for something else. They might want to reference it in another project or just appreciate the imagery. Since they asked specifically for the last line, I should make sure to present it clearly. Maybe they need it for a presentation, a quote, or to continue the poem. I should respond directly with the line and offer help if they need any

## Reference

1. https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/state.html